In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline  

In [4]:
data = pd.read_csv('data.dat', sep="   ", header=None).to_numpy().T
lab = pd.read_csv('label.dat', sep="   ", header=None).to_numpy()

/home/jfftilton/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/home/jfftilton/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [5]:
def initialization(X, K):
    m,n=X.shape
    pis = np.random.dirichlet(np.ones(K),size=1).reshape(K)
    
    idx = np.random.randint(n, size=n)
    mus = np.zeros(shape=(K,n))
    for k in range(K):
        idx = np.random.randint(m, size=int(m/K))
        x = X[idx]
        mu = x.mean(axis=0)
        mus[k] = mu
    sigmas = np.array([np.identity(n)]*K)
    return pis, mus, sigmas


def get_mus(X,expectations):
    n,m=X.shape
    K = expectations.shape[1]
    mus = np.zeros(shape=(K,m))
    for k in range(K):
        tau = expectations[:,k]
        numerator = np.zeros(shape=(m))
        for i in range(n):
            x_i=X[i,:]
            tau_i = tau[i]
            numerator += tau_i * x_i
        mu = numerator / np.sum(tau)
        mus[k] = mu
    return mus


def get_sigmas(X,expectations,mus):
    n,m=X.shape
    K = expectations.shape[1]
    sigmas = np.zeros((K, m, m))
    for k in range(K):
        tau = expectations[:,k]
        mu = mus[k,:]
        numerator = np.zeros(shape=(m,m))
        for i in range(n):
            x_i=X[i,:]
            tau_i = tau[i]
            x_i_minus_mu = (x_i - mu).reshape(m,1)
            numerator += tau_i*np.dot(x_i_minus_mu,x_i_minus_mu.T)
        sigmas[k] = numerator / np.sum(tau)
            
    return sigmas

In [ ]:
X = data[:5,:]
pis, mus, sigmas = initialization(X, K)
max_iter = 100
r=50

log_likelihood = []
for m in range(max_iter):
    K = 2
    m,n=X.shape
    ll = 0
    Es = np.zeros(shape=(m,K))
    Ns = np.zeros(shape=(m,K))
    for k in range(K):
        mu_k = mus[k,:]
        pi_k = pis[k]
        sigma_k = sigmas[k,:,:]
        exponents = np.zeros(shape=(m))
        N = np.zeros(shape=(m))
        for i in range(m):
            x = X[i,:]
            # decomposition
            e_val, e_vec = np.linalg.eigh(sigma_k)
            #order eigenvalue/vectors in descending order
            idx = np.argsort(e_val)[::-1]
            e_val = e_val[idx]
            e_vec = e_vec[:,idx]

            #threshold
            e_val = e_val[:r]
            e_vec = e_vec[:,:r]

            # Low rank approximation of values 
            sigma_approx = np.dot(np.dot(e_vec,np.diag(e_val**-1)),e_vec.T)
            mu_approx = np.dot(e_vec.T,mu)
            x_approx = np.dot(e_vec.T,x)
            x_approx_minus_mu_approx = x_approx - mu_approx
            exponent = np.exp(-.5 * np.sum((x_approx_minus_mu_approx**2) / e_val))
            denominator = np.sqrt(np.prod(e_val))
            ll += np.log(exponent) + np.log(denominator)
            exponents[i] = exponent
            en = exponent / denominator
            N[i] = en
        Es[:,k] = exponents
        Ns[:,k] = N
    log_likelihood.append(ll)
    print(ll)
    taus = Ns / np.sum(Ns)
    mus = get_mus(X,taus)
    pis = np.zeros(shape=(K))
    sigmas = get_sigmas(X,taus,mus)
    for k in range(K):
        tau_k = taus[:,k]
        pi_k = np.sum(tau_k) / m
        pis[k] = pi_k
    
        
       
    

            
            

In [ ]:
def get_approximations(x, mu, sigma, r=50):
    m,d = X.shape
    w,v = np.linalg.eigh(sigma)
    idx = np.argsort(w)[::-1]
    little_lam = w[:r]
    lambd = np.diag(little_lam**-1)
    U = v[:,:r]
    sigma_approx = np.dot(np.dot(U,lambd), U.T)
    mu_approx = np.dot(U.T, mu.reshape(d))
    x_approx = np.dot(U.T, x)
    return x_approx, mu_approx, sigma_approx, lambd, little_lam

def get_n_parts(x, mu, sigma, r=50):
    x_approx, mu_approx, sigma_approx, lambd, little_lam = get_approximations(x, mu, sigma, r=r)
    xm = x_approx-mu_approx
    denominator = np.prod(little_lam)
    likelihood = np.exp(np.dot(np.dot(xm.T,lambd), xm) * -.5)
    return likelihood, denominator


def get_mus(X,expectations):
    n,m=X.shape
    K = expectations.shape[1]
    mus = np.zeros(shape=(K,m))
    for k in range(K):
        tau = expectations[:,k]
        numerator = np.zeros(shape=(m))
        for i in range(n):
            x_i=X[i,:]
            tau_i = tau[i]
            numerator += tau_i * x_i
        mu = numerator / np.sum(tau)
        mus[k] = mu
    return mus

def get_sigmas(X,expectations,mus):
    n,m=X.shape
    K = expectations.shape[1]
    sigmas = np.zeros((K, m, m))
    for k in range(K):
        tau = expectations[:,k]
        mu = mus[k,:]
        numerator = np.zeros(shape=(m,m))
        for i in range(n):
            x_i=X[i,:]
            tau_i = tau[i]
            x_i_minus_mu = (x_i - mu).reshape(m,1)
            numerator += tau_i*np.dot(x_i_minus_mu,x_i_minus_mu.T)
        sigmas[k] = numerator / np.sum(tau)
            
    return sigmas


In [105]:
X = data[:100,:]
K = 2
pis, mus, sigmas = initialization(X, K)
m,d = X.shape
max_iter = 10
log_likelihoods = []
for z in range(max_iter):
    N = np.zeros(shape=(m, K))
    taus = np.zeros(shape=(m, K))
    k = 0
    log_likelihood = 0
    # e part
    for pi, mu, sigma in zip(pis,mus, sigmas):
        n_parts = np.apply_along_axis(get_n_parts, 1, X, mu, sigma, r=50)
        n = n_parts[:,0] / n_parts[:,1]
        tau = n * pi
        log_likelihood += np.sum(np.log(n_parts[:,0]))
        N[:,k] = n
        taus[:,k] = tau
        k+=1
    taus_sum = taus.sum(axis = 1)
    taus = np.array([taus[:,0]/taus_sum, taus[:,1]/taus_sum]).reshape(m,k)
    # m part
    pis = taus.sum(axis=0) / m
    mus = get_mus(X,taus)
    sigmas = get_sigmas(X,taus,mus)
    log_likelihoods.append(log_likelihood)
    print(log_likelihood)
    break

0.0


AxisError: axis 1 is out of bounds for array of dimension 1